In [2]:
from google.colab import drive

# Monta Google Drive en el entorno de Colab
drive.mount('/content/drive')

# Ruta al archivo de contraseñas JSON en drive
file_path = '/content/drive/MyDrive/pass.json'

# Lee el archivo JSON
with open(file_path, 'r') as file:
    config_data = json.load(file)

Mounted at /content/drive


In [3]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

# Acceder a las contraseñas de la api
api_key = config_data['api_key']

# Lista de nombres de países
paises = [
    'Afghanistan', 'Albania', 'Algeria', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Aruba', 'Austria',
    'Barbados', 'Belgium', 'Botswana', 'Brazil', 'Bulgaria', 'Canada', 'Colombia',
    'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Federated States of Micronesia', 'Finland',
    'France', 'Germany', 'Greece', 'Guyana', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Ireland', 'Israel',
    'Italy', 'Jamaica', 'Kenya', 'Kuwait', 'Latvia', 'Lithuania', 'Luxembourg', 'Madagascar', 'Malawi', 'Malta',
    'Marshall Islands', 'Mexico', 'Moldova', 'Mongolia', 'Montenegro', 'Myanmar', 'Netherlands', 'New Zealand',
    'North Macedonia', 'Norway', 'Oman', 'Palau', 'Papua New Guinea', 'Paraguay', 'Philippines', 'Poland',
    'Portugal', 'Romania', 'Russia', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Samoa', 'Senegal', 'Serbia',
    'Slovakia', 'Slovenia', 'Solomon Islands', 'Spain', 'Suriname', 'Sweden', 'Switzerland',
    'Tanzania', 'Thailand', 'Tonga', 'Trinidad and Tobago', 'United Kingdom of Great Britain and Northern Ireland',
    'United States', 'Vanuatu', 'Zimbabwe'
]

# Lista para almacenar los datos de cada país
all_data = []
# Lista para almacenar los nombres de los países
countries = []

# Iterar sobre la lista de países
for pais in paises:
    # Hacer la solicitud a la API para cada país
    url = f'https://api.openweathermap.org/data/2.5/weather?q={pais}&units=metric&lang=es&appid={api_key}'
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Convertir la respuesta a JSON y agregar los datos al listado
        data_json = json.loads(response.text)
        all_data.append({
            'main': data_json['main'],
            'weather': data_json['weather'][0]['description'],
            'clouds': data_json['clouds'],
            'wind': data_json['wind'],
            'visibility' : data_json['visibility'],
            'coord': data_json['coord'],
            'timezone': data_json['timezone'],
            'name': data_json['name']
        })
        countries.append(pais)
    else:
        print(f"No se pudo obtener datos para {pais}. Código de estado: {response.status_code}")

# Convertir la lista de datos a un DataFrame
df = pd.json_normalize(all_data, sep='_')
column_order = ['name', 'weather', 'main_temp', 'main_feels_like', 'main_temp_min', 'main_temp_max',
                'main_pressure', 'main_humidity', 'main_sea_level', 'main_grnd_level', 'clouds_all',
                'wind_speed', 'wind_deg', 'wind_gust','visibility','coord_lon',	'coord_lat','timezone']
df = df[column_order]

# Convertir el cambio de horario de segundos a timestamp UTC
for idx, i in enumerate(df['timezone']):
    fecha_hora = datetime.utcnow() + timedelta(seconds=i)
    fecha_hora_formateada = fecha_hora.strftime("%Y/%m/%d %H:%M")
    fecha_hora_formateada = pd.to_datetime(fecha_hora_formateada)
    df.at[idx, 'timezone'] = fecha_hora_formateada
# Mostrar el DataFrame
df

,name,weather,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_humidity,main_sea_level,main_grnd_level,clouds_all,wind_speed,wind_deg,wind_gust,visibility,coord_lon,coord_lat,timezone
0,Afghanistan,nubes,-0.07,-3.09,-0.07,-0.07,1021,41,1021.0,776.0,100,2.49,39,2.61,10000,66.0000,33.0000,2024-02-06 18:25:00
1,Albania,nubes,15.46,14.50,15.46,15.46,1023,55,1023.0,1012.0,100,1.77,270,1.71,10000,20.0000,41.0000,2024-02-06 14:55:00
2,Algeria,nubes,21.41,19.92,21.41,21.41,1025,12,1025.0,967.0,93,1.92,143,1.81,10000,3.0000,28.0000,2024-02-06 14:55:00
3,Anguilla,algo de nubes,29.00,34.53,28.86,29.45,1010,79,NaN,NaN,20,9.26,180,NaN,10000,-63.0500,18.2167,2024-02-06 09:55:00
4,Antigua and Barbuda,nubes dispersas,29.32,29.58,27.69,29.32,1012,46,NaN,NaN,40,5.14,170,11.83,10000,-61.8000,17.0500,2024-02-06 09:55:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Trinidad and Tobago,nubes,29.08,35.88,29.08,29.08,1014,84,1014.0,1014.0,92,9.15,126,12.21,10000,-61.0000,11.0000,2024-02-06 09:55:00
77,United Kingdom,nubes,6.30,1.82,6.30,6.37,1002,73,1002.0,993.0,100,8.15,279,14.18,10000,-2.6953,54.7584,2024-02-06 13:55:00
78,United States of America,nubes,3.02,-0.12,3.02,3.02,1020,78,1020.0,954.0,98,3.32,167,7.01,10000,-98.5000,39.7600,2024-02-06 07:55:00
79,Vanuatu,nubes dispersas,28.32,32.24,28.32,28.32,1003,76,1003.0,1003.0,45,1.99,62,1.91,10000,167.0000,-16.0000,2024-02-07 00:55:00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             81 non-null     object 
 1   weather          81 non-null     object 
 2   main_temp        81 non-null     float64
 3   main_feels_like  81 non-null     float64
 4   main_temp_min    81 non-null     float64
 5   main_temp_max    81 non-null     float64
 6   main_pressure    81 non-null     int64  
 7   main_humidity    81 non-null     int64  
 8   main_sea_level   61 non-null     float64
 9   main_grnd_level  61 non-null     float64
 10  clouds_all       81 non-null     int64  
 11  wind_speed       81 non-null     float64
 12  wind_deg         81 non-null     int64  
 13  wind_gust        66 non-null     float64
 14  visibility       81 non-null     int64  
 15  coord_lon        81 non-null     float64
 16  coord_lat        81 non-null     float64
 17  timezone         8

In [6]:
import psycopg2
from sqlalchemy import create_engine

# Conecta con Redshift
redshift_credentials = config_data['redshift_credentials'] # busca del archivo pass.json montado
conn = psycopg2.connect(
    dbname=redshift_credentials['dbname'],
    user=redshift_credentials['user'],
    password=redshift_credentials['password'],
    host=redshift_credentials['host'],
    port=redshift_credentials['port']
)

# Crea un motor para SQLAlchemy
engine = create_engine(f"postgresql://{redshift_credentials['user']}:{redshift_credentials['password']}@{redshift_credentials['host']}:{redshift_credentials['port']}/{redshift_credentials['dbname']}")

# Define la estructura de la tabla
tabla_sql = '''
DROP TABLE IF EXISTS weather_data;
CREATE TABLE weather_data (
    name VARCHAR(100),
    weather VARCHAR(100),
    main_temp DOUBLE PRECISION,
    main_feels_like DOUBLE PRECISION,
    main_temp_min DOUBLE PRECISION,
    main_temp_max DOUBLE PRECISION,
    main_pressure INT,
    main_humidity INT,
    main_sea_level DOUBLE PRECISION,
    main_grnd_level DOUBLE PRECISION,
    clouds_all INT,
    wind_speed DOUBLE PRECISION,
    wind_deg INT,
    wind_gust DOUBLE PRECISION,
    visibility INT,
    coord_lon DOUBLE PRECISION,
    coord_lat DOUBLE PRECISION,
    timezone TIMESTAMP
);
'''
# Ejecuta la consulta SQL para crear la tabla
conn.cursor().execute(tabla_sql)
conn.commit()

# Crea un cursor
cursor = conn.cursor()

for index, row in df.iterrows():
    # Consulta SQL para la inserción con todas las columnas
    sql_insert = '''
    INSERT INTO weather_data (
        name, weather, main_temp, main_feels_like, main_temp_min,
        main_temp_max, main_pressure, main_humidity, main_sea_level,
        main_grnd_level, clouds_all, wind_speed, wind_deg, wind_gust,
        visibility, coord_lon, coord_lat, timezone
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''

    # Extrae los valores de cada fila
    values = (
        row['name'], row['weather'], row['main_temp'], row['main_feels_like'],
        row['main_temp_min'], row['main_temp_max'], row['main_pressure'],
        row['main_humidity'], row['main_sea_level'], row['main_grnd_level'],
        row['clouds_all'], row['wind_speed'], row['wind_deg'], row['wind_gust'],
        row['visibility'], row['coord_lon'], row['coord_lat'], row['timezone']
    )

    # Ejecuta la consulta SQL con los valores de la fila actual
    cursor.execute(sql_insert, values)

# Confirma la transacción
conn.commit()

# Cierra el cursor y la conexión
cursor.close()
conn.close()